In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from DarcySolver import solve_darcy_on_doughnut_brain

T = 2000           # final time
num_steps = 50    # number of time steps
dt = T / num_steps

resolutions = [10, 50, 100]    # resolution
brain_radius = 100  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

kappas = [v*1e6 for v in [ 1e-8, 1e-10, 1e-12, 1e-14, 1e-16, 1e-18]]
kappas = [v*1e6 for v in [ 1e-8, 1e-12, 1e-18]]


visc = 0.8*1e-9     # in GPa*ms viscocity 
K_values = [kappa/visc for kappa in kappas]      # hydraulic conductivity

mmHg2GPa = 133.32*1e-9

A = 10*mmHg2GPa
f = 1 * 1e3 # in 1/ms


In [ ]:
# create mesh
meshes = []
for N in resolutions:
    mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))
    meshes.append(mesh)

probe_points = [
                Point(ventricle_radius + 0.2*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.4*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.6*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.8*(brain_radius - ventricle_radius), 0)
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]

p_obs = Expression("A*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*sin(2*pi*f*t)",
                   A=A,f=f,t=0,R_vent=ventricle_radius,degree=2)

p_skull = p_obs
p_ventricle = p_obs

f = Constant(0.0)

solutions = []
for i, kappa in enumerate(kappas):
    K = kappa/visc
    solutions.append([])
    for mesh in meshes:
        result = solve_darcy_on_doughnut_brain(mesh, f, T, num_steps, K=K,
                                                probe_points=probe_points,
                                                slice_points=slice_points,
                                                dirichlet_boundary_skull=p_skull,
                                                dirichlet_boundary_ventricle=p_ventricle)
        solutions[i].append(result)

In [ ]:
times = [200, 400, 600, 800]

idx = []
for i in range(num_steps):
    if np.isclose((i + 1)*dt, times).any():
        idx.append(i)

for i, kappa in enumerate(kappas):
    K = kappa/visc
    fig, axes = plt.subplots(nrows=1, ncols=len(meshes), figsize=(15,6))
    for j, res in enumerate(resolutions):
        cross_section = solutions[i][j]["slice"]
        plt.subplot(1,len(resolutions),j + 1)
        for n in idx:
            plt.plot(x_coords, cross_section[n,:]/mmHg2GPa,
                     label="t = {} ms".format((n +1)*dt))
        plt.legend()
        plt.grid()
        plt.xlabel("x in mm")
        plt.ylabel("p in mmHg")
        #plt.title(f"kappa={kappa:.1e},   h_min = {mesh.hmin():.2e}")
        plt.title(f"kappa={kappa:.2e},   N = {res}")


In [ ]:
# time refinement
N = 50
mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))

num_time_steps = [10, 50, 100]
time_ref_solutions = []
for i, kappa in enumerate(kappas):
    K = kappa/visc
    time_ref_solutions.append([])
    for num_steps in num_time_steps:
        result = solve_darcy_on_doughnut_brain(mesh, f, T, num_steps, K=K,
                                                probe_points=probe_points,
                                                slice_points=slice_points,
                                                dirichlet_boundary_skull=p_skull,
                                                dirichlet_boundary_ventricle=p_ventricle)
        time_ref_solutions[i].append(result)

In [ ]:
times = [0.4, 0.8, 1.2, 1.6, 2.0]


for i, kappa in enumerate(kappas):
    K = kappa/visc
    fig, axes = plt.subplots(nrows=1, ncols=len(meshes), figsize=(15,6))
    for j, num_steps in enumerate(num_time_steps):
        idx = []
        dt = T/num_steps
        for n in range(num_steps):
            if np.isclose((n + 1)*dt, times).any():
                idx.append(n)
        cross_section = time_ref_solutions[i][j]["slice"]
        plt.subplot(1,len(num_time_steps),j + 1)
        for n in idx:
            plt.plot(x_coords, cross_section[n,:]/mmHg2GPa,
                     label="t = {}".format((n +1)*dt))
        plt.legend()
        plt.grid()
        plt.xlabel("x in m")
        plt.ylabel("p in mmHg")
        #plt.title(f"kappa={kappa:.1e},   h_min = {mesh.hmin():.2e}")
        plt.title(f"kappa={kappa:.1e},   dt = {T/num_steps:.2f}")